要确定适当的训练轮次（epochs），并通过多次训练来评估模型的收敛情况，可以将现有代码拆分为多个模块，以便进行系统化的实验和分析。

以下是一个模块化的代码结构，包含数据加载、模型定义、训练与验证、实验管理以及结果可视化等部分。

# 1. 导入必要的库
首先，确保导入所有需要的库。

In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import clip
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import seaborn as sns

# 设置Seaborn风格
sns.set(style="darkgrid")


# 2. 设置设备和随机种子
为了确保实验的可重复性，设置随机种子，并选择计算设备（GPU或CPU）。

In [16]:
# 1. 设置设备和随机种子
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 设置随机种子
set_seed(42)

# 选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")


使用设备: cuda


# 3. 定义数据集类
将数据集相关的代码封装到一个类中，以便复用。

In [17]:
# 2. 定义数据集类
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, 
                 transform=None, partition=0):
        """
        初始化数据集
        :param img_dir: 图像文件夹路径
        :param attr_path: 属性文件路径
        :param bbox_path: 边界框文件路径
        :param partition_path: 分区文件路径
        :param transform: 图像预处理
        :param partition: 使用的数据分区 (0: train, 1: val, 2: test)
        """
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        attr_df = pd.read_csv(attr_path, sep=',', header=0)
        partition_df = pd.read_csv(partition_path, sep=',', header=0)
        
        # 合并属性文件和分区文件
        attr_df = attr_df.merge(partition_df, on='image_id')
        
        # 根据指定的分区进行筛选
        self.attr_df = attr_df[attr_df['partition'] == partition]
        
        # 读取边界框文件
        bbox_df = pd.read_csv(bbox_path, sep=',', header=0)
        
        # 合并边界框信息
        self.attr_df = self.attr_df.merge(bbox_df, on='image_id')

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像文件名
        img_name = self.attr_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        
        # 打开图像并转换为RGB
        image = Image.open(img_path).convert('RGB')
        
        # 获取属性标签 
        attrs = self.attr_df.iloc[idx, 1:41].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(np.float32)
        attrs = torch.tensor(attrs)
        
        # 应用图像预处理
        if self.transform:
            image = self.transform(image)
        
        return image, attrs


注意：在上述代码中，clip_emb 被随机初始化为标准正态分布。

你需要将其替换为实际的 CLIP 文本嵌入。具体来说，可以在训练循环外预先计算并传入相应的 CLIP 嵌入，或者在 Dataset 类中将其作为一个额外的返回值。

# 4. 定义CLIP文本嵌入生成函数
预先计算文本提示词的CLIP嵌入，以提高效率。

In [18]:
# 3. 定义 CLIP 文本嵌入生成函数
def generate_text_embeddings(clip_model, text_prompts, device):
    text_tokens = clip.tokenize(text_prompts).to(device)
    with torch.no_grad():
        text_embeddings = clip_model.encode_text(text_tokens)
    return text_embeddings.cpu()  # 移动到CPU


# 5. 定义CVAE模型
将模型定义封装为一个类。

In [19]:
class ClipCVAE(nn.Module):
    def __init__(self, img_channels=3, img_size=64, latent_dim=128, 
                 cond_dim=40, clip_dim=512):
        super(ClipCVAE, self).__init__()
        self.img_size = img_size
        self.latent_dim = latent_dim
        self.cond_dim = cond_dim
        self.clip_dim = clip_dim

        # 编码器部分
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels + cond_dim + clip_dim, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc_mu = nn.Linear(512*4*4, latent_dim)
        self.fc_logvar = nn.Linear(512*4*4, latent_dim)

        # 解码器部分
        self.decoder_input = nn.Linear(latent_dim + cond_dim + clip_dim, 512*4*4)
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x, c, clip_embedding):
        # 将条件标签、图像和CLIP嵌入拼接
        c = c.view(c.size(0), self.cond_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        clip_embedding = clip_embedding.view(clip_embedding.size(0), self.clip_dim, 1, 1).repeat(1, 1, self.img_size, self.img_size)
        x = torch.cat([x, c, clip_embedding], dim=1)
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z, c, clip_embedding):
        # 将潜在变量、条件标签和CLIP嵌入拼接
        z = torch.cat([z, c, clip_embedding], dim=1)
        x = self.decoder_input(z)
        x = x.view(-1, 512, 4, 4)
        x = self.decoder(x)
        return x

    def forward(self, x, c, clip_embedding):
        mu, logvar = self.encode(x, c, clip_embedding)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z, c, clip_embedding)
        return recon_x, mu, logvar


# 6. 定义损失函数
使用重构损失和KL散度作为损失函数。

In [20]:
# 5. 定义损失函数
def loss_function(recon_x, x, mu, logvar):
    criterion = nn.MSELoss(reduction='sum')
    recon_loss = criterion(recon_x, x)
    KL = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + KL


# 7. 定义训练和验证函数
封装训练和验证的逻辑。

In [21]:
# 6. 定义训练和验证函数
def train_one_epoch(model, train_loader, optimizer, device, CLIP_TEXT_EMBEDDINGS):
    model.train()
    train_loss = 0
    for batch_idx, (data, attrs) in enumerate(tqdm(train_loader, desc="训练中")):
        data = data.to(device, non_blocking=True)
        attrs = attrs.to(device, non_blocking=True)
        
        # 随机选择一个CLIP文本嵌入
        random_idx = torch.randint(0, len(CLIP_TEXT_EMBEDDINGS), (data.size(0),))
        clip_emb = CLIP_TEXT_EMBEDDINGS[random_idx].to(device)
        
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data, attrs, clip_emb)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    avg_loss = train_loss / len(train_loader.dataset)
    return avg_loss

def validate(model, val_loader, device, CLIP_TEXT_EMBEDDINGS):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, attrs in tqdm(val_loader, desc="验证中"):
            data = data.to(device, non_blocking=True)
            attrs = attrs.to(device, non_blocking=True)
            
            # 随机选择一个CLIP文本嵌入
            random_idx = torch.randint(0, len(CLIP_TEXT_EMBEDDINGS), (data.size(0),))
            clip_emb = CLIP_TEXT_EMBEDDINGS[random_idx].to(device)
            
            recon_batch, mu, logvar = model(data, attrs, clip_emb)
            loss = loss_function(recon_batch, data, mu, logvar)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader.dataset)
    return avg_val_loss


# 8. 定义模型训练函数
封装整个训练过程，包括多个epoch的训练和验证。

In [22]:
def train_model(model, train_loader, val_loader, optimizer, device, CLIP_TEXT_EMBEDDINGS, num_epochs=50):
    history = {'train_loss': [], 'val_loss': []}
    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}")
        train_loss = train_one_epoch(model, train_loader, optimizer, device, CLIP_TEXT_EMBEDDINGS)
        val_loss = validate(model, val_loader, device, CLIP_TEXT_EMBEDDINGS)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        print(f"训练损失: {train_loss:.4f}, 验证损失: {val_loss:.4f}")
    return history


# 9. 定义实验运行函数
用于多次训练相同轮次和不同轮次的实验。


In [23]:
# 8. 定义实验运行函数
def run_experiment(
    img_dir,
    attr_path,
    bbox_path,
    partition_path,
    text_prompts,
    device,
    num_runs=5,
    num_epochs=20,
    batch_size=128,
    learning_rate=1e-3
):
    # 图像预处理
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    
    # 创建训练集和验证集
    train_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                                  transform=transform, partition=0)
    val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                                transform=transform, partition=1)
    
    # 数据加载器（将 num_workers 设置为 0 以避免多进程问题）
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    
    # 加载CLIP模型并预先计算文本嵌入
    clip_model, preprocess_clip = clip.load("ViT-B/32", device=device)
    CLIP_TEXT_EMBEDDINGS = generate_text_embeddings(clip_model, text_prompts, device)
    
    # 记录所有运行的损失
    all_histories = []
    
    for run in range(1, num_runs + 1):
        print(f"\n运行 {run}/{num_runs}")
        
        # 每次运行前重置随机种子以确保独立性
        set_seed(42 + run)  # 不同运行使用不同的种子
        
        # 初始化模型、优化器
        latent_dim = 128
        cond_dim = 40
        clip_dim = 512
        model = ClipCVAE(img_channels=3, img_size=64, latent_dim=latent_dim, 
                        cond_dim=cond_dim, clip_dim=clip_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        # 训练模型
        history = train_model(model, train_loader, val_loader, optimizer, device, CLIP_TEXT_EMBEDDINGS, num_epochs=num_epochs)
        all_histories.append(history)
    
    return all_histories


# 10. 定义可视化函数
用于绘制多个训练运行的损失曲线以及不同轮次的比较。

In [24]:
# 9. 定义可视化函数
def plot_loss_curves(histories, num_epochs, title="Loss Curves"):
    """
    绘制多个训练运行的损失曲线，并显示平均值和标准差
    :param histories: 包含多个训练运行的历史损失
    :param num_epochs: 训练的总轮次
    :param title: 图表标题
    """
    train_losses = np.array([history['train_loss'] for history in histories])
    val_losses = np.array([history['val_loss'] for history in histories])
    
    # 计算平均值和标准差
    train_mean = train_losses.mean(axis=0)
    train_std = train_losses.std(axis=0)
    val_mean = val_losses.mean(axis=0)
    val_std = val_losses.std(axis=0)
    
    epochs = range(1, num_epochs + 1)
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_mean, label='training loss', color='blue')
    plt.fill_between(epochs, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.2)
    
    plt.plot(epochs, val_mean, label='verification loss', color='orange')
    plt.fill_between(epochs, val_mean - val_std, val_mean + val_std, color='orange', alpha=0.2)
    
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


# 11. 运行实验
现在，您可以运行实验，进行多次训练并绘制损失曲线。

## 11.1. 设置数据集路径和文本提示词
请根据实际路径修改数据集的路径。

In [25]:
# 数据集路径设置（请根据实际路径修改）
img_dir = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
attr_path = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
bbox_path = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
partition_path = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'

# 文本提示词
TEXT_PROMPTS = [
    "A portrait of a young woman",
    "A realistic face with a smile",
    "A person with distinct facial features"
]


# 11.2. 多次训练相同轮次
例如，进行5次训练，每次训练20个轮次。

In [12]:
# 训练参数
num_runs = 5
num_epochs = 20
batch_size = 128
learning_rate = 1e-3

# 运行实验
histories = run_experiment(
    img_dir=img_dir,
    attr_path=attr_path,
    bbox_path=bbox_path,
    partition_path=partition_path,
    text_prompts=TEXT_PROMPTS,
    device=device,
    num_runs=num_runs,
    num_epochs=num_epochs,
    batch_size=batch_size,
    learning_rate=learning_rate
)

# 可视化结果
plot_loss_curves(histories, num_epochs, title="Loss curves for multiple training sessions (same rounds)")



运行 1/5

Epoch 1/20


训练中:  99%|█████████▊| 1255/1272 [04:52<00:03,  4.29it/s]


KeyboardInterrupt: 

# 11.3. 探索最佳训练轮次
尝试不同的轮次设置，并进行多次训练以比较其表现。

In [ ]:
def hyperparameter_search(
    img_dir,
    attr_path,
    bbox_path,
    partition_path,
    text_prompts,
    device,
    epoch_settings=[10, 20, 30, 40, 50],
    num_runs=3,
    batch_size=128,
    learning_rate=1e-3
):
    all_histories = {}
    
    for num_epochs in epoch_settings:
        print(f"\n开始训练，轮次: {num_epochs}")
        histories = run_experiment(
            img_dir=img_dir,
            attr_path=attr_path,
            bbox_path=bbox_path,
            partition_path=partition_path,
            text_prompts=text_prompts,
            device=device,
            num_runs=num_runs,
            num_epochs=num_epochs,
            batch_size=batch_size,
            learning_rate=learning_rate
        )
        all_histories[num_epochs] = histories
    
    return all_histories

# 定义不同的轮次设置
epoch_settings = [10, 20, 30, 40, 50]
num_runs = 3

# 运行超参数搜索
all_histories = hyperparameter_search(
    img_dir=img_dir,
    attr_path=attr_path,
    bbox_path=bbox_path,
    partition_path=partition_path,
    text_prompts=TEXT_PROMPTS,
    device=device,
    epoch_settings=epoch_settings,
    num_runs=num_runs,
    batch_size=batch_size,
    learning_rate=learning_rate
)

# 可视化所有轮次的损失曲线
for num_epochs, histories in all_histories.items():
    plot_loss_curves(histories, num_epochs, title=f"epoch {num_epochs} of loss function")




开始训练，轮次: 10

运行 1/3

Epoch 1/10


训练中:  50%|████▉     | 635/1272 [00:32<00:37, 17.03it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>Traceback (most recent call last):

  File "/root/miniconda3/envs/pytorc_tes

训练损失: 1165.9932, 验证损失: 632.8052

Epoch 2/10


训练中:  59%|█████▉    | 756/1272 [00:39<00:25, 20.30it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
训练中:  64%|██████▍   | 818/1272 [00:42<00:21, 20.74it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/tor

训练损失: 565.8770, 验证损失: 523.7147

Epoch 3/10


...

训练损失: 462.0196, 验证损失: 459.0587

Epoch 8/20


训练中:   9%|▉         | 114/1272 [00:06<00:52, 22.08it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0><function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):

  File "/root/miniconda3/envs/pytorc_tes

训练损失: 455.7603, 验证损失: 456.8414

Epoch 9/20


训练中:   9%|▉         | 114/1272 [00:06<00:54, 21.21it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>
Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
Exception ignored in:  <function _MultiProcessingDataLoaderIter.__del__ at 0x7fa8f40622a0>  
 Traceback (most recent call last):
  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
       self._shutdown_workers()^
^^  File "/root/miniconda3/envs/pytorc_test1/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    ^if w.is_alive():^
^ ^ ^ ^ ^ ^ ^ 
^  File "/root/miniconda3/envs/pytorc_te

KeyboardInterrupt: 

# 12. 保存和加载模型
在训练完成后，您可以保存模型，并在需要时加载进行推理或进一步训练。

In [ ]:
# # 保存模型
# torch.save(model.state_dict(), 'clip_cvae_celeba.pth')
# print("模型已保存为 'clip_cvae_celeba.pth'")

In [29]:
# 加载模型
model = ClipCVAE(img_channels=3, img_size=64, latent_dim=128, 
                cond_dim=40, clip_dim=512).to(device)
model.load_state_dict(torch.load('cvae_celeba.pth', map_location=device))
model.eval()
print("模型已加载并设置为评估模式")


/tmp/ipykernel_12777/343482866.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cvae_celeba.pth', map_location=device))


RuntimeError: Error(s) in loading state_dict for ClipCVAE:
	size mismatch for encoder.0.weight: copying a param with shape torch.Size([64, 43, 4, 4]) from checkpoint, the shape in current model is torch.Size([64, 555, 4, 4]).
	size mismatch for decoder_input.weight: copying a param with shape torch.Size([8192, 168]) from checkpoint, the shape in current model is torch.Size([8192, 680]).

# 13. 生成图像并可视化
定义生成图像的函数，并进行可视化。

In [26]:
def generate_images(model, attrs, CLIP_TEXT_EMBEDDINGS, device, num_images=16):
    """
    根据条件标签和文本提示生成图像
    """
    model.eval()
    
    with torch.no_grad():
        z = torch.randn(num_images, model.latent_dim).to(device)
        attrs = attrs[:num_images].to(device)
        
        # 对于每个图像随机选择一个文本嵌入
        random_idx = torch.randint(0, len(CLIP_TEXT_EMBEDDINGS), (num_images,))
        text_embeddings = CLIP_TEXT_EMBEDDINGS[random_idx].to(device)
        
        generated = model.decode(z, attrs, text_embeddings)
        generated = generated.cpu()
        return generated

def show_images(images, title="Generated Images"):
    images = images * 0.5 + 0.5  # 反归一化
    grid = torchvision.utils.make_grid(images, nrow=4)
    np_grid = grid.numpy()
    plt.figure(figsize=(8,8))
    plt.imshow(np.transpose(np_grid, (1, 2, 0)))
    plt.title(title)
    plt.axis('off')
    plt.show()


生成和展示图像示例：

In [27]:
# 加载验证集
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
val_dataset = CelebADataset(img_dir, attr_path, bbox_path, partition_path, 
                            transform=transform, partition=1)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)

# 获取一批验证集数据
data_iter = iter(val_loader)
images, attrs = next(data_iter)
sample_attrs = attrs[:16]

# 生成图像
generated_images = generate_images(model, sample_attrs, CLIP_TEXT_EMBEDDINGS, device, num_images=16)

# 可视化生成的图像
show_images(generated_images, title="CLIP引导的条件生成人脸图像")


NameError: name 'model' is not defined